In [29]:
import os
import sys
import json

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
import findspark 

findspark.init()
findspark.find()
from pyspark.sql import SparkSession, Window
from pyspark.sql.types import StructType, StringType, StructField, IntegerType, TimestampType, DoubleType
from pyspark.sql.functions import from_json, col, udf, trim, regexp_replace, when, count,max,min,avg,sum,countDistinct, rank
from pyspark import SparkContext, SparkConf
spark = SparkSession.builder \
    .appName("join Data") \
    .getOrCreate()

In [1]:
import os
import sys
import json


In [2]:
with open("declaraciones.json", 'r', encoding='utf-8') as archivo:
    # Carga el contenido del archivo JSON en una variable
        datos_dependencias = json.load(archivo)

print(datos_dependencias)


with open("declaraciones.json", 'r', encoding='utf-8') as archivo:
    # Carga el contenido del archivo JSON en una variable
        datos_transformar= json.load(archivo)

[{'clasificador': 'Desconocido', 'dependencia': 'COORDINACION REGIONAL DE IDENTIFICACIÓN HUMANA', 'total': '10', 'url': 'http://www.declaranetcoahuila.gob.mx:9001/sidp/publica/.?q=204&q2=204'}, {'clasificador': 'Desconocido', 'dependencia': 'SEGURO POPULAR', 'total': '37', 'url': 'http://www.declaranetcoahuila.gob.mx:9001/sidp/publica/.?q=168&q2=168'}, {'clasificador': 'Desconocido', 'dependencia': 'SIN DEPENDENCIA', 'total': '4', 'url': 'http://www.declaranetcoahuila.gob.mx:9001/sidp/publica/.?q=0&q2=0'}, {'clasificador': 'Desconocido', 'dependencia': '', 'total': '16', 'url': 'http://www.declaranetcoahuila.gob.mx:9001/sidp/publica/.?q=SEDER&q2=SEDER'}, {'clasificador': 'Desconocido', 'dependencia': ' SELECCIONAR DEPENDENCIA', 'total': '34', 'url': 'http://www.declaranetcoahuila.gob.mx:9001/sidp/publica/.?q=16&q2=16'}, {'clasificador': 'UAdeC', 'dependencia': 'UNIVERSIDAD AUTONOMA DE COAHUILA', 'total': '9', 'url': 'http://www.declaranetcoahuila.gob.mx:9001/sidp/publica/.?q=200&q2=200

In [3]:
municipales = []
for e in datos_dependencias:
    if e["clasificador"] == "MUNICIPIOS":
        municipales.append(e["dependencia"])
print(municipales)

['ABASOLO', 'ACUÑA', 'ALLENDE', 'ARTEAGA', 'CANDELA', 'CASTAÑOS', 'CUATROCIENEGAS', 'ESCOBEDO', 'FRANCISCO I. MADERO', 'FRONTERA', 'GENERAL CEPEDA', 'GUERRERO', 'HIDALGO', 'JIMENEZ', 'JUAREZ', 'LAMADRID', 'MATAMOROS', 'MONCLOVA', 'MORELOS', 'MUZQUIZ', 'NADADORES', 'NAVA', 'OCAMPO', 'PARRAS', 'PIEDRAS NEGRAS', 'PROGRESO', 'RAMOS ARIZPE', 'SABINAS', 'SACRAMENTO', 'SALTILLO', 'SAN BUENAVENTURA', 'SAN JUAN DE SABINAS', 'SAN PEDRO', 'SIERRA MOJADA', 'TORREON', 'VIESCA', 'VILLA UNION', 'ZARAGOZA']


In [4]:
carpeta = "./data/"

archivos_json = [os.path.join(carpeta, archivo) for archivo in os.listdir(carpeta) if archivo.endswith('.json')]
df_final = None

# Iterar sobre cada archivo JSON y unirlo al DataFrame final
datos_final = []
datos_total = 0
for e in archivos_json:
    
        with open(e, 'r', encoding='utf-8') as archivo:
        # Carga el contenido del archivo JSON en una variable
            datos_json = json.load(archivo)
        datos_total += len(datos_json)
        print(f"{e} : {len(datos_json)} // {datos_total}")
        dependencia = e[7:-5]
        if dependencia.startswith("ZZZ"):
            dependencia =dependencia[3:] 
        for i in datos_json:
            if dependencia in municipales:
                try:        
    
 
                    i["metadata"]["institucion"] = dependencia
                    i["declaracion"]["situacionPatrimonial"]["datosEmpleoCargoComision"]["nombreEntePublico"] = dependencia
                    i["declaracion"]["situacionPatrimonial"]["datosEmpleoCargoComision"]["nivelOrdenGobierno"] = "MUNICIPAL_ALCALDIA"
                    
                except Exception as e:
                    print(e)
            else:
                i["metadata"]["institucion"] = dependencia
                i["declaracion"]["situacionPatrimonial"]["datosEmpleoCargoComision"]["nombreEntePublico"] = dependencia
            datos_final.append(i)
            
           
print(len(datos_final))
# with open('declaraciones_totales_v2.json', 'w', encoding='utf-8') as file:
#     json.dump(datos_final, file, ensure_ascii=False, indent=4)

./data/ABASOLO.json : 349 // 349
./data/ACUÑA.json : 3892 // 4241
./data/ADMINISTRACION FISCAL GENERAL.json : 2739 // 6980
./data/AGUAS Y SANEAMIENTO SABINAS.json : 18 // 6998
./data/ALLENDE.json : 874 // 7872
./data/ARCHIVO GENERAL DEL ESTADO.json : 61 // 7933
./data/ARTEAGA.json : 1356 // 9289
./data/AUDITORIA SUPERIOR DEL ESTADO DE COAHUILA.json : 1488 // 10777
./data/CANDELA.json : 139 // 10916
./data/CASTAÑOS.json : 250 // 11166
./data/CENTRO CULTURAL VITO ALESSIO ROBLES.json : 15 // 11181
./data/CENTRO DE JUSTICIA Y EMPODERAMIENTO PARA LAS MUJERES.json : 257 // 11438
./data/CERTTUC.json : 125 // 11563
./data/COLEGIO DE BACHILLERES DE COAHUILA.json : 1966 // 13529
./data/COLEGIO DE EDUCACION PROFESIONAL TECNICA DEL ESTADO DE COAHUILA.json : 553 // 14082
./data/COLEGIO DE ESTUDIOS CIENTIFICOS Y TECNOLOGICOS.json : 2363 // 16445
./data/COMISION COAHUILENSE DE CONCILIACION ARBITRAJE MEDICO.json : 14 // 16459
./data/COMISION COAHUILENSE DE CONCILIACION Y ARBITRAJE MEDICO.json : 33 // 